In [132]:
import pandas as pd
a1 = pd.read_csv('/home/cr625/a4/articles1.csv',index_col=0)
a2 = pd.read_csv('/home/cr625/a4/articles2.csv',index_col=0)
a3 = pd.read_csv('/home/cr625/a4/articles3.csv',index_col=0)

df = pd.concat([a1,a2,a3])

del a1, a2, a3


In [133]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [171]:
from tqdm import tqdm

frames = []
for i in tqdm(range(1000)):
    doc = df.loc[i,'content']
    text_id = df.loc[i,'id']
    doc = nlp(doc)
    ents = [(e.text, e.start_char, e.end_char, e.label_)
            for e in doc.ents
            if len(e.text.strip(' -—')) > 0]
    frame = pd.DataFrame(ents)
    frame['id'] = text_id
    frames.append(frame)

npf = pd.concat(frames)

npf.columns = ['Text','Start','Stop','Type','Id']

# collect occurrences law entities
law_related = npf[npf['Type'] == 'LAW']


100%|██████████| 1000/1000 [01:46<00:00,  9.39it/s]


In [174]:
from spacy.matcher import Matcher
from spacy import displacy

# only need one reference for each article
law_related_articles = law_related.groupby(['Id']).size()


patterns = [[{"LOWER":"facebook"}],[{"LOWER":"meta"}],[{"LOWER":"apple"}],[{"LOWER":"amazon"}],[{"LOWER":"netflix"}],[{"LOWER":"google"}]]
matcher = Matcher(nlp.vocab)
matcher.add("FAANG", patterns)

# collect all the articles that contain mentions of the FAANG entities
faang_related = []
for article in law_related_articles.index:
    doc = df.loc[article, 'content']
    matches = matcher(nlp(doc))
    if len(matches) > 0:
        faang_related.append(article)
        
# only need one copy of the article if the article has multiple mentions of the FAANG entities
article_list = pd.unique(faang_related)

for i in tqdm(range(len(article_list))):
    print(df.loc[article_list[i], 'title'])

100%|██████████| 29/29 [00:00<00:00, 23991.09it/s]

U.S.-Backed ‘Moderate’ Syrian Rebels: Beheading Boy Was a ’Mistake’
‘Orange is the New Black’ Star Sues NYPD for $10M After Arrest
Donald Trump Tombstone Appears in Central Park as Online Threats Continue - Breitbart
New York’s Left Protests the Peaceful Transfer of Power to Donald Trump - Breitbart
Gamescom Bans Cosplay Weapons in Response to Recent Terrorist Attacks - Breitbart
Gawker Ceases Operations, Posts Goodbyes - Breitbart
#ImWithAmber: Fans Call for Boycott of Johnny Depp’s Sweden Rock Concert over Abuse Allegations - Breitbart
Texas Veterinarian Loses License for Killing Cat with Bow
Microsoft Sued for $10,000 After Windows 10 Auto Update Renders Woman’s PC Unusable - Breitbart
Watch: SNL Portrays Smug Dems Election Night - Then Let Down After Trump Wins - Breitbart
Rand Paul Supports Trump as Nominee: ‘I Stand by That Pledge’
Wikileaks: Clinton Donors Say America ’Obviously Not a Democracy’ - Breitbart
Hillary Appointed Clinton Foundation Donor, Financial Bundler to Sensiti